## Getting Data from hitbtc Api

In [64]:
pip install requests 

In [65]:
import requests
import pandas as pd

In [66]:
#1. Caliin
response = requests.get("https://api.hitbtc.com/api/2/public/symbol")
response

#2.
response1 = requests.get("https://api.hitbtc.com/api/2/public/ticker")
response1

#3. 
response2 = requests.get("https://api.hitbtc.com/api/2/public/trades/ETHBTC")
response2

#4. 
response3 = requests.get("https://api.hitbtc.com/api/2/public/candles/ETHBTC?period=M30")
response3

<Response [200]>

## Converting Json Data to Dataframe


In [67]:

import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    return (text)

#jprint(response.json())

In [68]:
# converting json to dataframe.
df1 = pd.read_json(jprint(response.json()))
df2 = pd.read_json(jprint(response1.json()))
df3 = pd.read_json(jprint(response2.json()))
df4 = pd.read_json(jprint(response3.json()))

In [69]:
# list of data frames
lst = [df1, df2, df3, df4]

In [70]:
# concating all data
df_main = pd.concat(lst, axis = 1)

In [71]:
df_main

,baseCurrency,feeCurrency,id,provideLiquidityRate,quantityIncrement,quoteCurrency,takeLiquidityRate,tickSize,marginTrading,maxInitialLeverage,ask,bid,high,last,low,open,symbol,timestamp,volume,volumeQuote,id,price,quantity,side,timestamp,close,max,min,open,timestamp,volume,volumeQuote
0,FTM,USD,FTMUSD,0.001,1.0000,USD,0.0025,1.000000e-07,NaN,NaN,0.274175,0.273694,0.287629,0.273165,0.272022,0.291000,FTMUSD,2021-06-16 14:38:54.343000+00:00,7.127000e+03,2046.683637,1.289077e+09,0.062464,22.0870,buy,2021-06-16 14:38:48.265000+00:00,0.006100,0.006500,0.005000,0.006000,2015-08-20 19:00:00+00:00,0.515,0.003127
1,SYBC,USD,SYBCUSD,0.001,0.1000,USD,0.0025,1.000000e-06,NaN,NaN,0.000002,0.000001,0.000002,0.000001,0.000001,0.000002,SYBCUSD,2021-06-16 14:38:51+00:00,1.208536e+10,12190.621909,1.289077e+09,0.062464,0.0400,buy,2021-06-16 14:38:42.712000+00:00,0.006110,0.006300,0.005880,0.006100,2015-08-20 19:30:00+00:00,0.470,0.002887
2,STEEM,BTC,STEEMBTC,0.001,0.1000,BTC,0.0025,1.000000e-09,NaN,NaN,0.000011,0.000010,0.000011,0.000010,0.000010,0.000011,STEEMBTC,2021-06-16 14:38:54.548000+00:00,6.061859e+05,6.462049,1.289077e+09,0.062456,6.5593,sell,2021-06-16 14:38:35.779000+00:00,0.006280,0.006280,0.006280,0.006280,2015-08-20 20:00:00+00:00,0.023,0.000144
3,PPC,USD,PPCUSD,0.001,0.1000,USD,0.0025,1.000000e-06,NaN,NaN,1.152351,1.133030,1.177945,1.176482,1.175238,1.174509,PPCUSD,2021-06-16 14:38:52.944000+00:00,2.701000e+02,317.807207,1.289077e+09,0.062461,0.1561,buy,2021-06-16 14:38:35.774000+00:00,0.006280,0.006280,0.006280,0.006280,2015-08-20 22:00:00+00:00,0.041,0.000257
4,REEF,USD,REEFUSD,0.001,1.0000,USD,0.0025,1.000000e-07,NaN,NaN,0.021500,0.021450,0.022537,0.021336,0.021183,0.023183,REEFUSD,2021-06-16 14:38:53.139000+00:00,1.831480e+05,4067.875242,1.289077e+09,0.062457,23.5276,buy,2021-06-16 14:38:21.450000+00:00,0.006291,0.006291,0.006291,0.006291,2015-08-21 00:00:00+00:00,0.004,0.000025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,POWR,ETH,POWRETH,0.001,0.1000,ETH,0.0025,1.000000e-08,NaN,NaN,0.000081,0.000080,0.000080,0.000080,0.000073,0.000073,POWRETH,2021-06-16 14:38:52.645000+00:00,3.699000e+02,0.029618,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1236,MKR,DAI,MKRDAI,0.001,0.0001,DAI,0.0025,1.000000e-04,NaN,NaN,3017.215900,3002.660100,3107.522200,3107.522200,3107.522200,3107.522200,MKRDAI,2021-06-16 14:38:53.656000+00:00,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1237,SUPER,USD,SUPERUSD,0.001,0.0100,USD,0.0025,1.000000e-05,NaN,NaN,0.584470,0.582530,0.646940,0.583980,0.581460,0.646940,SUPERUSD,2021-06-16 14:38:52.841000+00:00,5.451820e+03,3381.401612,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1238,CHSB,BTC,CHSBBTC,0.001,10.0000,BTC,0.0025,1.000000e-10,NaN,NaN,0.000017,0.000017,0.000018,0.000017,0.000017,0.000018,CHSBBTC,2021-06-16 14:38:54.651000+00:00,2.429750e+06,42.567653,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN


## Data Preprocessing

In [72]:
# removing duplicate colums
df_main = df_main.loc[:,~df_main.columns.duplicated()]

In [73]:
lst_drop = ["quantityIncrement", "takeLiquidityRate", "feeCurrency",
            "marginTrading","maxInitialLeverage", "ask","bid", "id", "price", "quantity", "close"]
# dropping the columns 
df_main = df_main.drop(lst_drop, axis = 1)
df_main

,baseCurrency,provideLiquidityRate,quoteCurrency,tickSize,high,last,low,open,symbol,timestamp,volume,volumeQuote,side,max,min
0,FTM,0.001,USD,1.000000e-07,0.287629,0.273165,0.272022,0.291000,FTMUSD,2021-06-16 14:38:54.343000+00:00,7.127000e+03,2046.683637,buy,0.006500,0.005000
1,SYBC,0.001,USD,1.000000e-06,0.000002,0.000001,0.000001,0.000002,SYBCUSD,2021-06-16 14:38:51+00:00,1.208536e+10,12190.621909,buy,0.006300,0.005880
2,STEEM,0.001,BTC,1.000000e-09,0.000011,0.000010,0.000010,0.000011,STEEMBTC,2021-06-16 14:38:54.548000+00:00,6.061859e+05,6.462049,sell,0.006280,0.006280
3,PPC,0.001,USD,1.000000e-06,1.177945,1.176482,1.175238,1.174509,PPCUSD,2021-06-16 14:38:52.944000+00:00,2.701000e+02,317.807207,buy,0.006280,0.006280
4,REEF,0.001,USD,1.000000e-07,0.022537,0.021336,0.021183,0.023183,REEFUSD,2021-06-16 14:38:53.139000+00:00,1.831480e+05,4067.875242,buy,0.006291,0.006291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,POWR,0.001,ETH,1.000000e-08,0.000080,0.000080,0.000073,0.000073,POWRETH,2021-06-16 14:38:52.645000+00:00,3.699000e+02,0.029618,NaN,NaN,NaN
1236,MKR,0.001,DAI,1.000000e-04,3107.522200,3107.522200,3107.522200,3107.522200,MKRDAI,2021-06-16 14:38:53.656000+00:00,0.000000e+00,0.000000,NaN,NaN,NaN
1237,SUPER,0.001,USD,1.000000e-05,0.646940,0.583980,0.581460,0.646940,SUPERUSD,2021-06-16 14:38:52.841000+00:00,5.451820e+03,3381.401612,NaN,NaN,NaN
1238,CHSB,0.001,BTC,1.000000e-10,0.000018,0.000017,0.000017,0.000018,CHSBBTC,2021-06-16 14:38:54.651000+00:00,2.429750e+06,42.567653,NaN,NaN,NaN


In [74]:
# renaming the columns
df_main=df_main.rename(columns = {'baseCurrency':'Base currency quantity traded', 'quoteCurrency': 'Quote currency quantity traded',
                           'provideLiquidityRate': 'Liquidity in pool','last':'close' })

In [75]:
# rearranging colums
x=list(df_main.columns)
x

['Base currency quantity traded',
 'Liquidity in pool',
 'Quote currency quantity traded',
 'tickSize',
 'high',
 'close',
 'low',
 'open',
 'symbol',
 'timestamp',
 'volume',
 'volumeQuote',
 'side',
 'max',
 'min']

In [76]:
x=['timestamp','side','Base currency quantity traded','Quote currency quantity traded','open','high','low','close','volume','Liquidity in pool','tickSize','max','min','volumeQuote']

In [77]:
df_new=pd.DataFrame()

In [78]:
for i in x:
  df_new[i]=df_main[i]
df_new

,timestamp,side,Base currency quantity traded,Quote currency quantity traded,open,high,low,close,volume,Liquidity in pool,tickSize,max,min,volumeQuote
0,2021-06-16 14:38:54.343000+00:00,buy,FTM,USD,0.291000,0.287629,0.272022,0.273165,7.127000e+03,0.001,1.000000e-07,0.006500,0.005000,2046.683637
1,2021-06-16 14:38:51+00:00,buy,SYBC,USD,0.000002,0.000002,0.000001,0.000001,1.208536e+10,0.001,1.000000e-06,0.006300,0.005880,12190.621909
2,2021-06-16 14:38:54.548000+00:00,sell,STEEM,BTC,0.000011,0.000011,0.000010,0.000010,6.061859e+05,0.001,1.000000e-09,0.006280,0.006280,6.462049
3,2021-06-16 14:38:52.944000+00:00,buy,PPC,USD,1.174509,1.177945,1.175238,1.176482,2.701000e+02,0.001,1.000000e-06,0.006280,0.006280,317.807207
4,2021-06-16 14:38:53.139000+00:00,buy,REEF,USD,0.023183,0.022537,0.021183,0.021336,1.831480e+05,0.001,1.000000e-07,0.006291,0.006291,4067.875242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,2021-06-16 14:38:52.645000+00:00,NaN,POWR,ETH,0.000073,0.000080,0.000073,0.000080,3.699000e+02,0.001,1.000000e-08,NaN,NaN,0.029618
1236,2021-06-16 14:38:53.656000+00:00,NaN,MKR,DAI,3107.522200,3107.522200,3107.522200,3107.522200,0.000000e+00,0.001,1.000000e-04,NaN,NaN,0.000000
1237,2021-06-16 14:38:52.841000+00:00,NaN,SUPER,USD,0.646940,0.646940,0.581460,0.583980,5.451820e+03,0.001,1.000000e-05,NaN,NaN,3381.401612
1238,2021-06-16 14:38:54.651000+00:00,NaN,CHSB,BTC,0.000018,0.000018,0.000017,0.000017,2.429750e+06,0.001,1.000000e-10,NaN,NaN,42.567653


In [79]:
# saving the result in csv
df_new.to_csv('out.csv')